In [ ]:
import openai
import requests
import re
import chardet

In [ ]:
def generate_prompt(repository, code):
    prompt = f"""
    Generate a code complexity score for the following code snippet:
    --------------------------------------------------
    Code:
    {code}
    --------------------------------------------------

    """
    return prompt


In [ ]:

# Use GPT-3 to analyze the code
def analyze_code(prompts):
    response = openai.Completion.create(
        engine="ada",
        prompt=prompts,
        max_tokens=100,
        temperature=0.7,
        n=len(prompts),
        stop=None
    )
    complexity_scores = []
    for choice in response.choices:
        score = extract_complexity_score(choice.text)
        if score is not None:
            complexity_scores.append(score)
        else:
            complexity_scores.append(0)
    return complexity_scores



In [ ]:
def extract_complexity_score(text):
    # Extract the complexity score using a regular expression
    pattern = r"Complexity Score: (\d+)"
    match = re.search(pattern, text)
    if match:
        complexity_score = int(match.group(1))
        return complexity_score
    else:
        return None



In [ ]:
# def identify_most_complex_repository(repositories):
#     complexity_scores = {}

#     for repository in repositories:
#         preprocessed_contents = preprocess_code(repository)
#         if preprocessed_contents:
#             prompts = []
#             for content in preprocessed_contents:
#                 prompt = generate_prompt(repository, content)
#                 prompts.append(prompt)

#             scores = analyze_code(prompts)
#             avg_score = sum(scores) / len(scores)
#             complexity_scores[repository["name"]] = avg_score

#     most_complex_repository = max(complexity_scores, key=complexity_scores.get)
#     return most_complex_repository

# def main():
#     st.title("Code Complexity Analyzer")

#     st.write("Enter your GitHub username to analyze your repositories:")
#     github_url = st.text_input("GitHub URL", "")

#     if st.button("Analyze"):
#         st.write("Analyzing repositories...")

#         repositories = get_user_repositories(github_url)

#         if repositories:
#             most_complex_repository = identify_most_complex_repository(repositories)

#             st.write("Analysis complete!")
#             st.write(f"The most complex repository is: {most_complex_repository}")
#         else:
#             st.write("No repositories found.")

# if __name__ == "__main__":
#     main()

def identify_most_complex_repository(repositories):
    complexity_scores = {}

    for repository in repositories:
        preprocessed_contents = preprocess_code(repository)
        if preprocessed_contents:
            prompts = []
            for content in preprocessed_contents:
                prompt = generate_prompt(repository, content)
                prompts.append(prompt)

            scores = analyze_code(prompts)
            complexity_score = sum(scores) / len(scores)
            complexity_scores[repository["name"]] = complexity_score

    if complexity_scores:
        most_complex_repository = max(complexity_scores, key=complexity_scores.get)
        justification = generate_justification(most_complex_repository)
        most_complex_repo_url = None
        for repo in repositories:
            if repo["name"] == most_complex_repository:
                most_complex_repo_url = repo["url"]
                break
        return most_complex_repository, complexity_scores[most_complex_repository], justification, most_complex_repo_url
    else:
        return None, 0, "", None



In [ ]:

def generate_justification(repository):
    prompt = f"Justify why the repository '{repository}' is considered the most complex:"

    response = openai.Completion.create(
        engine="ada",
        prompt=prompt,
        max_tokens=100,
        temperature=0.7,
        n=1,
        stop=None
    )

    if response.choices:
        justification = response.choices[0].text.strip()
        return justification
    else:
        return ""